In [1]:
import pandas as pd
import plotly.graph_objects as go

# ====================================
# File paths (adjust path if needed)
# ====================================
base_path = "./"

files = {
    "Bridge": base_path + "converted_coordinates_Resultat_Bridge.csv",
    "RailJoint": base_path + "converted_coordinates_Resultat_RailJoint.csv",
    "Turnout": base_path + "converted_coordinates_Turnout.csv"
}

# ====================================
# Marker styles for each category
# ====================================
marker_styles = {
    "Bridge": {"color": "red", "size": 10},
    "RailJoint": {"color": "blue", "size": 8},
    "Turnout": {"color": "green", "size": 12}
}

# ====================================
# Load and process data
# ====================================
data_frames = []

for category, file in files.items():
    try:
        df = pd.read_csv(file, encoding="utf-8")
        df.columns = df.columns.str.strip()

        if "Latitude" in df.columns and "Longitude" in df.columns:
            df["Latitude"] = pd.to_numeric(df["Latitude"], errors="coerce")
            df["Longitude"] = pd.to_numeric(df["Longitude"], errors="coerce")
            df = df[["Latitude", "Longitude"]]
            df["Category"] = category
            data_frames.append(df)
            print(f"{category} data loaded. Rows: {len(df)}")
        else:
            print(f"{category} file is missing 'Latitude' or 'Longitude' columns.")
            print("Available columns:", df.columns.tolist())
    except Exception as e:
        print(f"Error loading {category} file:", e)

if not data_frames:
    raise ValueError("No valid data found. Please check the CSV files.")

data = pd.concat(data_frames, ignore_index=True)
data = data.dropna(subset=["Latitude", "Longitude"])

print("Data loaded for categories:")
print(data["Category"].value_counts())
print("Summary of Latitude/Longitude:")
print(data.describe()[["Latitude", "Longitude"]])
print("Missing values:")
print(data.isnull().sum())

# ====================================
# Plotting map
# ====================================
fig = go.Figure()

for category, style in marker_styles.items():
    category_data = data[data["Category"] == category]

    if len(category_data) > 0:
        fig.add_trace(go.Scattermapbox(
            lat=category_data["Latitude"],
            lon=category_data["Longitude"],
            mode="markers",
            marker=dict(color=style["color"], size=style["size"]),
            name=category
        ))
    else:
        print(f"No data for category: {category}")

fig.update_layout(
    mapbox_style="open-street-map",
    title="Railway Infrastructure Map (Bridges, Joints, Turnouts)",
    legend_title_text="Legend",
    width=1200,
    height=800,
    margin={"r": 0, "t": 50, "l": 50, "b": 0},
    mapbox=dict(
        zoom=10,
        center=dict(lat=data["Latitude"].mean(), lon=data["Longitude"].mean())
    )
)

if len(fig.data) == 0:
    print("No traces added to the map.")

# Show map
fig.show()


Bridge data loaded. Rows: 25
RailJoint data loaded. Rows: 20
Turnout data loaded. Rows: 75
Data loaded for categories:
Category
Turnout      75
Bridge       25
RailJoint    20
Name: count, dtype: int64
Summary of Latitude/Longitude:
         Latitude   Longitude
count  120.000000  120.000000
mean    60.792431   14.921160
std      0.191516    0.284255
min     60.510878   14.518605
25%     60.587961   14.570193
50%     60.748120   15.064924
75%     61.007341   15.120486
max     61.009065   15.352741
Missing values:
Latitude     0
Longitude    0
Category     0
dtype: int64


/var/folders/y_/g4bb5p814w172ydvc3rlbn0r0000gn/T/ipykernel_87963/908989428.py:69: DeprecationWarning:

*scattermapbox* is deprecated! Use *scattermap* instead. Learn more at: https://plotly.com/python/mapbox-to-maplibre/

